# 使用selenium和bs4爬虫

## 1.selenium的使用

In [1]:
import os,time
from selenium import webdriver
from bs4 import BeautifulSoup
chromedriver = "C:\\Program Files (x86)\\Google\\Chrome\\Application\\chromedriver.exe"#chrome所在驱动
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get('http://user.100xuexi.com/MyDeal/myEBookPurchase.aspx')
time.sleep(2)

In [ ]:
driver.find_element_by_id('username').send_keys('18775325832')#

In [ ]:
driver.find_element_by_id('txtVerify').clear()#验证码
driver.find_element_by_id('txtVerify').send_keys('69')#验证码

In [ ]:
#driver.find_element_by_xpath('//*[@id="conPurchaseA1"]/table/tbody/tr[3]/td[2]/a').click();
driver.find_element_by_xpath('//*[@id="regNow"]').click();

In [ ]:
driver.find_element_by_xpath('//*[@id="PurchaseA2"]').click();

In [2]:
#接下来会有新的窗口打开，获取所有窗口句柄
windows = driver.window_handles
driver.switch_to.window(windows[-1])

In [ ]:
driver.find_element_by_xpath('//*[@id="lefter"]/div/div[1]/a').click();
#/PracticeCenter/Chapter/Index?TQuestionPlanID=2997&groupName=&code=&tb_l_PaperQuePlanID=86215&Model=chapter&TypeMenuFlag=2
#/PracticeCenter/Chapter/Index?TQuestionPlanID=2997&groupName=&code=&tb_l_PaperQuePlanID=86216&Model=chapter&TypeMenuFlag=2
#item表示章，li后的数字表示节

In [ ]:
#driver.find_element_by_xpath('//*[@id="item01"]/li[2]/a').click();

## 2.解析成html

In [2120]:
html = driver.page_source
soup = BeautifulSoup(html, "html5lib")

## 3.html文件解析

### 1.解析问题

In [2121]:
import re

In [2122]:
baseurl_item = u'2Z104000  施工质量管理\\2Z104040　施工质量事故预防与处理'
single_baseurl = u'二级建造师anki\\二建2018管理\\'+ baseurl_item + u'\\单选\\'
multiple_baseurl = u'二级建造师anki\\二建2018管理\\'+ baseurl_item + u'\\多选\\'

In [2123]:
content_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [2124]:
#soup.find('div',class_='ItemBotCompose').decompose()

In [2125]:
# replace with `soup.findAll` if you are using BeautifulSoup3
for div in soup.find_all("div", {'class':'Item config'}): 
    if len(div.find_all("div", {'class':'ItemBotCompose'}))!=0:
        div.decompose() #将满足调节的 div 项删除

In [2126]:
single_content_all = []
multiple_content_all = []

single_content_all = content_all[0].find_all("div",attrs={"class":"question config"})
if len(content_all)>1:
    multiple_content_all = content_all[1].find_all("div",attrs={"class":"question config"})

In [2127]:
len(single_content_all)

22

单选

In [2128]:
single_question = []

for i in range(len(single_content_all)):
    single_contents = single_content_all[i].dt.contents
    #print(contents_single)
    single_s = ''
    for j in single_contents:#单选
        single_s = single_s+str(j)
    single_question.append(re.sub('[\n]','',single_s))

In [2129]:
print(len(single_question))

22


多选

In [2130]:
multiple_question = []

for i in range(len(multiple_content_all)):#多选
    multiple_contents = multiple_content_all[i].dt.contents
    #print(contents_single)
    multiple_s = ''
    for j in multiple_contents:
        multiple_s = multiple_s+str(j)
    multiple_question.append(re.sub('[\n]','',multiple_s))

In [2131]:
print(len(multiple_question))

13


保存数据

In [2132]:
#question_file = open('question_file.txt', 'w')
import codecs

single_folder = os.path.exists(single_baseurl)
multiple_folder = os.path.exists(multiple_baseurl)
if not single_folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
    os.makedirs(single_baseurl)
                
if not multiple_folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
    os.makedirs(multiple_baseurl)
                
single_question_file = codecs.open(single_baseurl + 'single_question.txt', 'w','utf-8')
multiple_question_file = codecs.open(multiple_baseurl + 'multiple_question.txt', 'w','utf-8')

for item in single_question:
    single_question_file.write("%s\r\n" % item)
    
for item in multiple_question:
    multiple_question_file.write("%s\r\n" % item)
    
single_question_file.close()
multiple_question_file.close()

### 2.解析选项

In [2133]:
option_all = soup.find_all("div",attrs={"class":"nodeItem"})

区别单选和多选

In [2134]:
single_option_all = option_all[0].find_all("div",attrs={"class":"question config"})
if len(option_all)>1:
    multiple_option_all = option_all[1].find_all("div",attrs={"class":"question config"})

In [2135]:
len(single_option_all)

22

In [2136]:
len(multiple_option_all)

13

单选

In [2137]:
single_option_a = []
single_option_b = []
single_option_c = []
single_option_d = []
for option_i in range(len(single_option_all)):
    single_option_a.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[0].get_text()))
    single_option_b.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[1].get_text()))
    single_option_c.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[2].get_text()))
    single_option_d.append(re.sub('[\r|\n|\t|\\s]', '', single_option_all[option_i].table.tbody.find_all("label")[3].get_text()))

多选

In [2138]:
multiple_option_a = []
multiple_option_b = []
multiple_option_c = []
multiple_option_d = []
multiple_option_e = []
for option_i in range(len(multiple_option_all)):
    multiple_option_a.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[0].get_text()))
    multiple_option_b.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[1].get_text()))
    multiple_option_c.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[2].get_text()))
    multiple_option_d.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[3].get_text()))
    multiple_option_e.append(re.sub('[\r|\n|\t|\\s]', '', multiple_option_all[option_i].table.tbody.find_all("label")[4].get_text()))

In [2139]:
single_option_a[1]

'A．一般事故'

保存单选选项

In [2140]:
import codecs

single_option_a_file = codecs.open(single_baseurl + 'single_option_a.txt', 'w','utf-8')
single_option_b_file = codecs.open(single_baseurl + 'single_option_b.txt', 'w','utf-8')
single_option_c_file = codecs.open(single_baseurl + 'single_option_c.txt', 'w','utf-8')
single_option_d_file = codecs.open(single_baseurl + 'single_option_d.txt', 'w','utf-8')

for item in single_option_a:
    single_option_a_file.write("%s\r\n" % item)
for item in single_option_b:
    single_option_b_file.write("%s\r\n" % item)
    
for item in single_option_c:
    single_option_c_file.write("%s\r\n" % item)
    
for item in single_option_d:
    single_option_d_file.write("%s\r\n" % item)

single_option_a_file.close()
single_option_b_file.close()
single_option_c_file.close()
single_option_d_file.close()

保存多选选项

In [2141]:
import codecs

multiple_option_a_file = codecs.open(multiple_baseurl + 'multiple_option_a.txt', 'w','utf-8')
multiple_option_b_file = codecs.open(multiple_baseurl + 'multiple_option_b.txt', 'w','utf-8')
multiple_option_c_file = codecs.open(multiple_baseurl + 'multiple_option_c.txt', 'w','utf-8')
multiple_option_d_file = codecs.open(multiple_baseurl + 'multiple_option_d.txt', 'w','utf-8')

for item in multiple_option_a:
    multiple_option_a_file.write("%s\r\n" % item)
for item in multiple_option_b:
    multiple_option_b_file.write("%s\r\n" % item)
    
for item in multiple_option_c:
    multiple_option_c_file.write("%s\r\n" % item)
    
for item in multiple_option_d:
    multiple_option_d_file.write("%s\r\n" % item)

multiple_option_a_file.close()
multiple_option_b_file.close()
multiple_option_c_file.close()
multiple_option_d_file.close()

### 解析答案

In [2142]:
anwser_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [2143]:
single_anwser_all = anwser_all[0].find_all("div",attrs={"class":"Item config"}) #单选
if len(anwser_all)>1:
    multiple_anwser_all = anwser_all[1].find_all("div",attrs={"class":"Item config"}) #多选

In [2144]:
len(single_anwser_all)

22

In [2145]:
single_answer_right = []
multiple_answer_right = []

for answer_i in range(len(single_anwser_all)):
    #print(anwser_all[25].find("div",attrs={"class":"config answer"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}).get_text())
    single_answer_right.append(single_anwser_all[answer_i].find("div",attrs={"class":"anser Con"}).string)
    
for answer_i in range(len(multiple_anwser_all)):
    #print(anwser_all[25].find("div",attrs={"class":"config answer"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}))
    #print(anwser_all[answer_i].find("table",attrs={"class":"HiddenStart"}).find("div",attrs={"class":"anser Con"}).get_text())
    multiple_answer_right.append(multiple_anwser_all[answer_i].find("div",attrs={"class":"anser Con"}).string)
    
len(single_answer_right)

22

In [2146]:
import codecs

single_anwser_file = codecs.open(single_baseurl + 'single_anwser.txt', 'w','utf-8')
multiple_anwser_file = codecs.open(multiple_baseurl + 'multiple_anwser.txt', 'w','utf-8')

for item in multiple_answer_right:
    multiple_anwser_file.write("%s\r\n" % item)
    
single_anwser_file.close()
multiple_anwser_file.close()

In [2147]:
#print(re.sub('[\r|\n|\t|\\s]', '', anwser_1))

### 解析解析

In [2148]:
analysis_all = soup.find_all("div",attrs={"class":"nodeItem"})

In [2149]:
single_analysis_all = analysis_all[0].find_all("div",attrs={"class":"Item config"}) #单选
if len(analysis_all)>1:
    multiple_analysis_all = analysis_all[1].find_all("div",attrs={"class":"Item config"}) #多选
len(single_analysis_all)

22

In [2150]:
single_analysis = []
multiple_analysis = []

for analysis_i in range(len(single_analysis_all)):
    print(single_analysis_all[analysis_i].find("div",attrs={"class":"analysits Con"}))
    print('\n')
    #answer_right.append(anwser_all[answer_i].find("div",attrs={"class":"analysits Con"}).string)
#print(answer_right)
for analysis_i in range(len(multiple_analysis_all)):
    print(multiple_analysis_all[analysis_i].find("div",attrs={"class":"analysits Con"}))
    print('\n')

<div class="analysits Con">重大质量事故是指造成10人以上30人以下死亡,或者50人以上100人以下重伤,或者5000万元以上1亿元以下直接经济损失的事故。</div>


<div class="analysits Con">《关于做好房屋建筑和市政基础设施工程质量事故报告和调查处理工作的通知》（建质[2010]111号）根据工程质量事故造成的人员伤亡或者直接经济损失，将工程质量事故分为4个等级：①特别重大事故，指造成30人以上死亡，或者100人以上重伤，或者1亿元以上直接经济损失的事故；②重大事故，指造成10人以上30人以下死亡，或者50人以上100人以下重伤，或者5000万元以上1亿元以下直接经济损失的事故；③较大事故，指造成3人以上10人以下死亡，或者10人以上50人以下重伤，或者1000万元以上5000万元以下直接经济损失的事故；④一般事故，指造成3人以下死亡，或者10人以下重伤，或者100万元以上1000万元以下直接经济损失的事故。本等级划分所称的“以上”包括本数，所称的“以下”不包括本数。据此，本题中的事故等级为一般事故。</div>


<div class="analysits Con">工程质量事故按事故责任分类可分为三种：①指导责任事故，指由于工程指导或领导失误而造成的质量事故。如由于工程负责人不按规范指导施工，强令他人违章作业，或片面追求施工进度，放松或不按质量标准进行控制和检验，降低施工质量标准等而造成的质量事故。②操作责任事故，指在施工过程中，由于操作者不按规程和标准实施操作，而造成的质量事故。如浇筑混凝土时随意加水，或振捣疏漏造成混凝土质量事故等。③自然灾害事故，指由于突发的严重自然灾害等不可抗力造成的质量事故。如地震、台风、暴雨、雷电及洪水等造成工程破坏甚至倒塌。</div>


<div class="analysits Con">质量事故按产生的原因分类，可分为：①技术原因引发的质量事故；②管理原因引发的质量事故；③社会、经济原因引发的质量事故；④其他原因引发的质量事故。其中，管理原因引发的质量事故是指管理上的不完善或失误引发的质量事故。例如，施工单位或监理单位的质量管理体系不完善，检验制度不严密，质量控制不严格，质量管理措施落实不力，检测仪器设备管理不善而失准，材料检验不严等原因引起的质量事故。BD两项

In [2151]:
print(multiple_analysis_all[0].find("div",attrs={"class":"analysits Con"}))

<div class="analysits Con">技术原因引发的工程质量事故是指在工程项目实施中由于设计、施工在技术上的失误而造成的质量事故。如结构设计计算错误；对地质情况估计错误；采用了不适宜的施工方法或施工工艺等引发质量事故。BCE三项属于引发质量事故的管理原因。</div>


In [2152]:
single_analay = []
multiple_analay = []
for i in range(len(single_analysis_all)):
    single_analay_item = single_analysis_all[i].find("div",attrs={"class":"analysits Con"}).contents
    #print(analay_single)
    single_analay.append(single_analay_item)
    #print(analay_single)
    
for i in range(len(multiple_analysis_all)):
    multiple_analay_item = multiple_analysis_all[i].find("div",attrs={"class":"analysits Con"}).contents
    #print(analay_single)
    multiple_analay.append(multiple_analay_item)
    #print(analay_single)

In [2153]:
single_analay_final = []
for item in single_analay:
    #analysis_file.write("%s" % item)
    single_s = ''
    for i in item:
        single_s = single_s + str(i)
    single_analay_final.append(single_s)
    #analysis_file.write("%s\n" % s)
#analysis_file.close()

multiple_analay_final = []
for item in multiple_analay:
    #analysis_file.write("%s" % item)
    multiple_s = ''
    for i in item:
        multiple_s = multiple_s + str(i)
    multiple_analay_final.append(multiple_s)
    #analysis_file.write("%s\n" % s)
#analysis_file.close()

In [2154]:
import codecs
single_analysis_file = codecs.open(single_baseurl + 'single_analysis_file.txt', 'w','utf-8')

for s in single_analay_final:
    print(s)
    print("\n")
    single_analysis_file.write("%s\r\n" % s)
    
multiple_analysis_file = codecs.open(multiple_baseurl + 'multiple_analysis_file.txt', 'w','utf-8')

for s in multiple_analay_final:
    print(s)
    print("\n")
    multiple_analysis_file.write("%s\r\n" % s)    
    
single_analysis_file.close()
multiple_analysis_file.close()

重大质量事故是指造成10人以上30人以下死亡,或者50人以上100人以下重伤,或者5000万元以上1亿元以下直接经济损失的事故。


《关于做好房屋建筑和市政基础设施工程质量事故报告和调查处理工作的通知》（建质[2010]111号）根据工程质量事故造成的人员伤亡或者直接经济损失，将工程质量事故分为4个等级：①特别重大事故，指造成30人以上死亡，或者100人以上重伤，或者1亿元以上直接经济损失的事故；②重大事故，指造成10人以上30人以下死亡，或者50人以上100人以下重伤，或者5000万元以上1亿元以下直接经济损失的事故；③较大事故，指造成3人以上10人以下死亡，或者10人以上50人以下重伤，或者1000万元以上5000万元以下直接经济损失的事故；④一般事故，指造成3人以下死亡，或者10人以下重伤，或者100万元以上1000万元以下直接经济损失的事故。本等级划分所称的“以上”包括本数，所称的“以下”不包括本数。据此，本题中的事故等级为一般事故。


工程质量事故按事故责任分类可分为三种：①指导责任事故，指由于工程指导或领导失误而造成的质量事故。如由于工程负责人不按规范指导施工，强令他人违章作业，或片面追求施工进度，放松或不按质量标准进行控制和检验，降低施工质量标准等而造成的质量事故。②操作责任事故，指在施工过程中，由于操作者不按规程和标准实施操作，而造成的质量事故。如浇筑混凝土时随意加水，或振捣疏漏造成混凝土质量事故等。③自然灾害事故，指由于突发的严重自然灾害等不可抗力造成的质量事故。如地震、台风、暴雨、雷电及洪水等造成工程破坏甚至倒塌。


质量事故按产生的原因分类，可分为：①技术原因引发的质量事故；②管理原因引发的质量事故；③社会、经济原因引发的质量事故；④其他原因引发的质量事故。其中，管理原因引发的质量事故是指管理上的不完善或失误引发的质量事故。例如，施工单位或监理单位的质量管理体系不完善，检验制度不严密，质量控制不严格，质量管理措施落实不力，检测仪器设备管理不善而失准，材料检验不严等原因引起的质量事故。BD两项属于技术原因；C项属于社会、经济原因。


质量事故按产生的原因分类，可分为：①技术原因引发的质量事故；②管理原因引发的质量事故；③社会、经济原因引发的质量事故；④其他原因引发的质量事故。其中，管理原因引发的质量事故是指管理上的不完善或失误引发的质量事

单选选项拼接

In [2155]:
"""
question
option_a
option_b
option_c
option_d
answer_right
analay_final
"""

"""
option= codecs.open(baseurl + 'option.txt', 'w','utf-8')

for i in range(len(option_a)):
    option.write(option_a[i].strip().replace(u'A\uff0e','')
                 +u'<br>'+option_b[i].strip().replace(u'B\uff0e','')
                 +u'<br>'+option_c[i].strip().replace(u'C\uff0e','')
                 +u'<br>'+option_d[i].replace(u'D\uff0e','')
                 )
"""
single_data = []
for i in range(len(single_option_a)):
    single_data_string = single_question[i].replace('（　　）','{{c1::（）}}') + "	"+ single_option_a[i].strip().replace('A．','')+ '<br>' + single_option_b[i].strip().replace('B．','')+ '<br>'+ single_option_c[i].strip().replace('C．','')+ '<br>'+ single_option_d[i].replace('D．','')+ "	"+ single_answer_right[i]+ "	"+ single_analay_final[i]
    single_data.append(single_data_string)
data_single = codecs.open(single_baseurl + 'data_single.txt', 'w','utf-8')

for i in range(len(single_data)):
    data_single.write(single_data[i]+"\r\n")
data_single.close()

In [2156]:
len(single_data )

22

多选选项拼接

In [2157]:
multiple_data = []
for i in range(len(multiple_option_a)):
    multiple_data_string = multiple_question[i].replace('（　　）','{{c1::（）}}') + "	"+ multiple_option_a[i].strip().replace('A．','')+ '<br>' + multiple_option_b[i].strip().replace('B．','')+ '<br>'+ multiple_option_c[i].strip().replace('C．','')+ '<br>'+ multiple_option_d[i].replace('D．','')+ '<br>'+ multiple_option_e[i].replace('E．','')+ "	"+ multiple_answer_right[i]+ "	"+ multiple_analay_final[i]
    multiple_data.append(multiple_data_string)
data_multiple = codecs.open(multiple_baseurl + 'data_multiple.txt', 'w','utf-8')

for i in range(len(multiple_data)):
    data_multiple.write(multiple_data[i]+"\r\n")
data_multiple.close()

In [2158]:
len(multiple_data)

13